# 互斥锁
- 抢占资源

In [2]:
%%writefile test_mutex.py
from multiprocessing import Process
import time

def task(name):
    print(f"{name} 1")
    time.sleep(1)
    print(f"{name} 2")
    time.sleep(1)
    print(f"{name} 3")

if __name__ =='__main__':
    # 共享控制台，谁抢到谁输出
    for i in range(3):
        p=Process(target=task,args=(f"进程{i}",))
        p.start()

            

Writing test_mutex.py


- 加互斥锁

In [1]:
%%writefile test_mutex.py
from multiprocessing import Process,Lock
import time

def task(name,mutex):
    mutex.acquire() #* 加锁！
    print(f"{name} 1")
    time.sleep(1)
    print(f"{name} 2")
    time.sleep(1)
    print(f"{name} 3")
    mutex.release() #* 解锁！

if __name__ =='__main__':
    # 主进程创建一把共享锁
    mutex=Lock()
    for i in range(3):
        #* 将共享锁传给子进程，确保大家公用一把锁
        p=Process(target=task,args=(f"进程{i}",mutex))
        p.start()
            

Overwriting test_mutex.py


# 抢票练习
 - 无锁

In [3]:
%%writefile test_mutex.py
from multiprocessing import Process 
import time
import json

def search(name):
    time.sleep(1)
    dic=json.loads(open("db.txt", "r"))
    print(f'{name} 查看剩余票数：{dic["count"]}')
    
def get(name):
    time.sleep(1)
    dic=json.loads(open("db.txt", "r"))
    if dic["count"] > 0:
        dic["count"]=dic["count"]-1
        print(f"{name} 购票成功")
        time.sleep(3)
        json.dump(dic, open("db.txt", "w"))
    else:
        print(f"{name} 购票失败")

def task(name):
    search(name)
    get(name)

if __name__ =="__main__":
    #* 无锁，都会抢票成功
    for i in range(10):
        p=Process(target=task,args=(f"路人{i}",))
        p.start()
    

Overwriting test_mutex.py


 - 加锁

In [1]:
%%writefile test_mutex.py
from multiprocessing import Process,Lock
import time
import json

def search(name):
    # 模拟网络延迟
    time.sleep(1)
    dic=json.load(open(r"00Pratice\03os\01多进程\db.txt", "r"))
    print(f'{name} 查看剩余票数：{dic["count"]}')
    
def get(name,mutex):
    time.sleep(1)
    mutex.acquire() #* 加锁
    dic=json.load(open(r"00Pratice\03os\01多进程\db.txt", "r"))
    if dic["count"] > 0:
        dic["count"]=dic["count"]-1
        print(f"{name} 购票成功")
        time.sleep(3)
        json.dump(dic, open(r"00Pratice\03os\01多进程\db.txt", "w"))
    else:
        print(f"{name} 购票失败")
    mutex.release() #* 解锁

def task(name,mutex):
    #! 查票操作不要加锁，影响效率。
    search(name)
    get(name,mutex)

if __name__ =="__main__":
    #* 加锁
    mutex=Lock()
    for i in range(10):
        p=Process(target=task,args=(f"路人{i}",mutex))
        p.start()
    

Overwriting test_mutex.py


 - 互斥锁与join的区别</br>
   join方法效率低，会变成单进程。</br>
   互斥锁可以提升效率，锁的粒度可以控制。


In [ ]:
%%writefile test_mutex.py
from multiprocessing import Process,Lock
import time
import json

def search(name):
    # 模拟网络延迟
    time.sleep(1)
    dic=json.load(open(r"00Pratice\03os\01多进程\db.txt", "r"))
    print(f'{name} 查看剩余票数：{dic["count"]}')
    
def get(name,mutex):
    time.sleep(1)
    dic=json.load(open(r"00Pratice\03os\01多进程\db.txt", "r"))
    if dic["count"] > 0:
        dic["count"]=dic["count"]-1
        print(f"{name} 购票成功")
        time.sleep(3)
        json.dump(dic, open(r"00Pratice\03os\01多进程\db.txt", "w"))
    else:
        print(f"{name} 购票失败")

def task(name):
    #! 查票操作不要加锁，影响效率。
    search(name)
    get(name)

if __name__ =="__main__":
    #* 加锁
    mutex=Lock()
    for i in range(10):
        p=Process(target=task,args=(f"路人{i}",))
        p.start()
        p.join()
    